Load Data from this competitions Link

https://www.kaggle.com/competitions/anomaly-detection/data?select=train.csv

In [ ]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: ar-AE,ar;q=0.9,en-US;q=0.8,en;q=0.7" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/29569/2386789/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1663216197&Signature=AWJ3jUB51ZzuV7CU1xgrklNmEJL%2FYjGdygvXG6UrQqXU2nOpmK%2BUYJorZXlZdMSfpdqy0yZo%2FKgbhzqao83RJK034Xc3UiEFZvx4NdZ8ByvdlSwApbADoIycuCjGXDZzwPpVq2Y4Rl2hjRqA5W9vK2GDdhIrcdS8hgSrtXrtamIerAs2wqa8kCcc%2BoKYoAVIxY%2F1Wa%2BguPUT72dZezAToge33FzJ7oasHxpBAGRQjYu1XJHIcK9OKp9tuoFue1CJS6bdo9rJRG2KQ0N27dLjtVWhkcVtiWh8DTeBgY7Z%2FgUET%2F4K6DE46BKLhIweGBXbC3f%2Fo%2Bjjh%2BDzM9bE48utPw%3D%3D&response-content-disposition=attachment%3B+filename%3Danomaly-detection.zip" -c -O 'anomaly-detection.zip'

--2022-09-12 04:31:08--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/29569/2386789/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1663216197&Signature=AWJ3jUB51ZzuV7CU1xgrklNmEJL%2FYjGdygvXG6UrQqXU2nOpmK%2BUYJorZXlZdMSfpdqy0yZo%2FKgbhzqao83RJK034Xc3UiEFZvx4NdZ8ByvdlSwApbADoIycuCjGXDZzwPpVq2Y4Rl2hjRqA5W9vK2GDdhIrcdS8hgSrtXrtamIerAs2wqa8kCcc%2BoKYoAVIxY%2F1Wa%2BguPUT72dZezAToge33FzJ7oasHxpBAGRQjYu1XJHIcK9OKp9tuoFue1CJS6bdo9rJRG2KQ0N27dLjtVWhkcVtiWh8DTeBgY7Z%2FgUET%2F4K6DE46BKLhIweGBXbC3f%2Fo%2Bjjh%2BDzM9bE48utPw%3D%3D&response-content-disposition=attachment%3B+filename%3Danomaly-detection.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 173.194.210.128, 173.194.211.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225974 (221K) [application/zip]
Saving to: ‘anomaly-detection.zip’

anom

Unzip data

In [ ]:
!unzip /content/anomaly-detection.zip

Archive:  /content/anomaly-detection.zip
  inflating: Submission.csv          
  inflating: test.csv                
  inflating: train.csv               


Read Dataset

In [ ]:
import pandas as pd

df = pd.read_csv('/content/train.csv')

df.head()

,timestamp,value,is_anomaly,predicted
0,1425008573,42,False,44.072500
1,1425008873,41,False,50.709390
2,1425009173,41,False,81.405120
3,1425009473,61,False,39.950367
4,1425009773,44,False,35.350160


In [ ]:
# drop column timestamp

df=df.drop(['timestamp'], axis=1)

In [ ]:
# make a conditions to split anomaly rows from non anomaly

is_anomaly_T=df["is_anomaly"]==True
is_anomaly_F=df["is_anomaly"]==False

In [ ]:
#split anomaly rows from non anomaly 

df_f=df[is_anomaly_F]
df_t=df[is_anomaly_T]

In [ ]:
df_f=df_f.drop(['is_anomaly'], axis=1)
df_t=df_t.drop(['is_anomaly'], axis=1)

In [ ]:
train_x=df_f.values
test_x=df_t.values

In [ ]:
train_x=train_x.astype('float32')
test_x=test_x.astype('float32')

In [ ]:
import cv2
import glob
import keras
from keras import layers
import numpy as np
import random
import tensorflow as tf
seed=42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

create our model

In [ ]:
import keras
from keras import layers


input_img = keras.Input(shape=(2,))

x1=layers.Dense(50,activation='relu')(input_img)
# "encoded" is the encoded representation of the input
encoded = layers.Dense(32, activation='relu')(x1)

x2=layers.Dense(50,activation='relu')(encoded)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(2)(x2)
autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
from keras.callbacks import ModelCheckpoint
filepath = 'my_best_model.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'

checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')

we here train our model with non anomaly data only
to make our model can reconstruct non anomaly data only with lowest possible error and at test time anomaly data will give us high error

In [ ]:
autoencoder.fit(train_x, train_x,
                epochs=50,
                batch_size=64,
                shuffle=True,
                validation_split=0.2,callbacks=[checkpoint])

Epoch 1/50
189/189 [==============================] - ETA: 0s - loss: 0.0676
Epoch 1: val_loss improved from inf to 0.19865, saving model to my_best_model.epoch01-loss0.20.hdf5
189/189 [==============================] - 0s 3ms/step - loss: 0.0676 - val_loss: 0.1986
Epoch 2/50
165/189 [=========================>....] - ETA: 0s - loss: 0.0301
Epoch 2: val_loss improved from 0.19865 to 0.03862, saving model to my_best_model.epoch02-loss0.04.hdf5
189/189 [==============================] - 0s 2ms/step - loss: 0.0273 - val_loss: 0.0386
Epoch 3/50
167/189 [=========================>....] - ETA: 0s - loss: 0.7489
Epoch 3: val_loss did not improve from 0.03862
189/189 [==============================] - 0s 2ms/step - loss: 1.2367 - val_loss: 2.1600
Epoch 4/50
185/189 [============================>.] - ETA: 0s - loss: 0.6743
Epoch 4: val_loss did not improve from 0.03862
189/189 [==============================] - 0s 2ms/step - loss: 0.6761 - val_loss: 0.2318
Epoch 5/50
177/189 [==================

In [ ]:
from keras.models import Sequential, load_model

model = load_model("/content/my_best_model.epoch02-loss0.04.hdf5")

In [ ]:
model.evaluate(model.predict(train_x),train_x)

471/471 [==============================] - 1s 1ms/step - loss: 0.0723


0.07231775671243668

Test with anomaly data give us high error

In [ ]:
model.evaluate(model.predict(test_x),test_x)

25/25 [==============================] - 0s 1ms/step - loss: 94.5257


94.52570343017578